In [2]:
#Expand notebook to take full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

#Jupyter magic to notify when a cell finishes execution with %%notify command -- does not work with Jupyterlab
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

###
import sys
sys.path.insert(0,'../../src/')

%load_ext autoreload
%autoreload 2

import copy
%matplotlib inline
import matplotlib.pyplot as plt
%pdb

<IPython.core.display.Javascript object>

Automatic pdb calling has been turned ON


In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.nn.modules import Module
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import copy
import pickle
import argparse

model_dir = "./models/"
import model_archs
from utils_plot import show_sws_weights, show_weights, print_dims, prune_plot, draw_sws_graphs, joint_plot, plot_data
from utils_model import test_accuracy, train_epoch, retrain_sws_epoch, model_prune, get_weight_penalty, layer_accuracy
from utils_misc import trueAfterN, logsumexp, root_dir, model_load_dir, get_ab, get_sparsity
from utils_sws import GaussianMixturePrior, special_flatten, KL, compute_responsibilies, merger, sws_prune, sws_prune_l2, sws_prune_copy, sws_replace, compressed_model
from mnist_loader import search_train_data, search_retrain_data, search_validation_data, train_data, test_data, batch_size
from extract_targets import get_targets, get_layer_data
from retrain_layer import retrain_layer
retraining_epochs = 50


test_data_full = Variable(test_data(fetch = "data")).cuda()
test_labels_full = Variable(test_data(fetch = "labels")).cuda()
#val_data_full = Variable(search_validation_data(fetch = "data")).cuda()
#val_labels_full = Variable(search_validation_data(fetch = "labels")).cuda()


In [4]:
layer_model_1, gmp_1, res_1 = retrain_layer(5000, 2, 5000, 10, 16, 1, 1e-4, 3, "search", "SWSModel", "MSEHNA", "")

0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -141128.50
Epoch: 50. Loss: -142871.94
Original: 98.73% - Retrain: 97.68% - Prune: 96.91% - Quantize: 96.65% - Sparsity: 97.68%
Original: 98.75% - Retrain: 97.97% - Prune: 97.01% - Quantize: 96.85% - Sparsity: 97.68%


In [5]:
def loadfile(mean, zmean, tau, temp, mixtures, model, data_size = "full",  scaling = False, model_save_dir = "", fn="", file = "res"):
    r = 30 if file == 'ures' else 50
    exp_name = "{}_m{}_zm{}_r{}_t{}_m{}_kdT{}_{}".format(model, mean, zmean, r, tau, int(mixtures), int(temp),data_size) + fn
    if (file=='res' or file == 'ures' or file == 'gmp'):
        with open(model_save_dir + '/mnist_retrain_layer_{}_{}.p'.format(file, exp_name),'rb') as f:
            file = pickle.load(f)
    if (file=='model'):
        file = torch.load(model_save_dir + '/mnist_retrain_layer_model_{}.m'.format(exp_name))
    return file



In [7]:
gmp_1 = loadfile(mean = 5000, zmean = 5000, tau = 1e-6, temp = 1, mixtures = 15, model = "LeNet_300_100FC1", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "gmp")

In [7]:
#1. Train all 3 layers
#2. Unify model and replace priors
#3. Optimize as usual
def load_layer_retrain_LeNet_300_100(mean, zmean, temp, tau, mixtures, data_size, loss_type = 'MSEHNA', savedir = ''):
    zvar = 10
    var = 2
    model_name = 'LeNet_300_100'
    
    #train 3 layers independently
    #layer_model_1, gmp_1, res_1 = retrain_layer(mean, var, zmean, zvar, mixtures, 1, tau, 1, data_size, model_name, loss_type, savedir)
    #layer_model_2, gmp_2, res_2 = retrain_layer(mean, var, zmean, zvar, mixtures, 1, tau, 2, data_size, model_name, loss_type, savedir)
    #layer_model_3, gmp_3, res_3 = retrain_layer(mean, var, zmean, zvar, mixtures, temp, tau, 3, data_size, model_name, loss_type, savedir)
    layer_model_1 = loadfile(mean = 5000, zmean = 5000, tau = 1e-7, temp = 1, mixtures = 15, model = "LeNet_300_100FC1", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "model")
    layer_model_2 = loadfile(mean = 5000, zmean = 5000, tau = 1e-5, temp = 1, mixtures = 15, model = "LeNet_300_100FC2", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "model")
    layer_model_3 = loadfile(mean = 5000, zmean = 5000, tau = 1e-4, temp = temp, mixtures = 15, model = "LeNet_300_100FC3", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "model")
    gmp_1 = loadfile(mean = 5000, zmean = 5000, tau = 1e-7, temp = 1, mixtures = 15, model = "LeNet_300_100FC1", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "gmp")
    gmp_2 = loadfile(mean = 5000, zmean = 5000, tau = 1e-5, temp = 1, mixtures = 15, model = "LeNet_300_100FC2", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "gmp")
    gmp_3 = loadfile(mean = 5000, zmean = 5000, tau = 1e-4, temp = temp, mixtures = 15, model = "LeNet_300_100FC3", data_size = 'search', scaling = False, model_save_dir = "./expfiles", file = "gmp")
    
    #transfer GMP to new unified model architecture
    unified_model = sws_replace(model_archs.LeNet_300_100().cuda(), [layer_model_1.state_dict(), layer_model_2.state_dict(), layer_model_3.state_dict()])
    gmp_u1 = GaussianMixturePrior(mixtures, [x for x in unified_model.fc1.parameters()], 0.99, zero_ab = get_ab(zmean, zvar), ab = get_ab(mean, var), means = gmp_1.means.clone().data.cpu().numpy(), scaling = False)
    gmp_u2 = GaussianMixturePrior(mixtures, [x for x in unified_model.fc2.parameters()], 0.99, zero_ab = get_ab(zmean, zvar), ab = get_ab(mean, var), means = gmp_2.means.clone().data.cpu().numpy(), scaling = False)
    gmp_u3 = GaussianMixturePrior(mixtures, [x for x in unified_model.fc3.parameters()], 0.99, zero_ab = get_ab(zmean, zvar), ab = get_ab(mean, var), means = gmp_3.means.clone().data.cpu().numpy(), scaling = False)

    #load datasets
    data_size = 'search'
    if(data_size == 'search'):
        train_dataset = search_train_data()
        val_data_full = Variable(test_data(fetch='data')).cuda()
        val_labels_full = Variable(test_data(fetch='labels')).cuda()
        un_val_acc = test_accuracy(val_data_full, val_labels_full, unified_model)[0]
    if(data_size == 'full'):
        train_dataset = train_data()

    loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)    
    
    #unified accuracy
    unify_res = plot_data(unified_model, mode = 'retrain', data_size = data_size, loss_type='CE', mv = (mean, var), zmv = (zmean, zvar), tau = tau, temp = temp, mixtures = mixtures)
    unify_res.data_epoch(0, unified_model)
    
    #prune-stats before retraining
    l1_prune = sws_prune_l2(dict((l, unified_model.state_dict()[l]) for l in ("fc1.bias", "fc1.weight")), gmp_u1)
    l2_prune = sws_prune_l2(dict((l, unified_model.state_dict()[l]) for l in ("fc2.bias", "fc2.weight")), gmp_u2)
    l3_prune = sws_prune_l2(dict((l, unified_model.state_dict()[l]) for l in ("fc3.bias", "fc3.weight")), gmp_u3)
    pruned_model = sws_replace(unified_model, [l1_prune, l2_prune, l3_prune])
    unify_res.data_prune(pruned_model)
    preprune = copy.deepcopy(unify_res.prune_acc)
    presparsity = unify_res.sparsity
    
    #init optimizers
    loss_type = 'CESNT'
    opt_1 = torch.optim.Adam([
            {'params': unified_model.fc1.parameters(), 'lr': 1e-4},
            {'params': [gmp_u1.means], 'lr': 3e-4},
            {'params': [gmp_u1.gammas, gmp_u1.rhos], 'lr': 3e-3}])
    opt_2 = torch.optim.Adam([
            {'params': unified_model.fc2.parameters(), 'lr': 1e-4},
            {'params': [gmp_u2.means], 'lr': 3e-4},
            {'params': [gmp_u2.gammas, gmp_u2.rhos], 'lr': 3e-3}])
    opt_3 = torch.optim.Adam([
            {'params': unified_model.fc3.parameters(), 'lr': 1e-4},
            {'params': [gmp_u3.means], 'lr': 3e-4},
            {'params': [gmp_u3.gammas, gmp_u3.rhos], 'lr': 3e-3}])

    #align and retrain
    show_sws_weights(unified_model)
    retraining_epochs=80
    for epoch in range(retraining_epochs):
        unified_model, loss = retrain_sws_epoch(unified_model, gmp_u1, opt_1, loader, 1e-7, 1, loss_type)
        unified_model, loss = retrain_sws_epoch(unified_model, gmp_u2, opt_2, loader, 1e-5, 1, loss_type)
        unified_model, loss = retrain_sws_epoch(unified_model, gmp_u3, opt_3, loader, 1e-4, 1, loss_type)
        #test_acc = test_accuracy(test_data_full, test_labels_full, model_n)
        #print('Epoch: {}. Test Accuracy: {:.2f}'.format(epoch+1, test_acc[0]))
        #model_n, means = clamp_weights (model_n, means)
        unify_res.data_epoch(epoch+1, unified_model)
        
        if (trueAfterN(epoch, 10)):
            print ("Epoch: {}, Test Acc: {:.2f}".format(epoch+1, unify_res.test_accuracy[-1]))
    show_sws_weights(unified_model)

    #prune model
    l1_prune = sws_prune_l2(dict((l, unified_model.state_dict()[l]) for l in ("fc1.bias", "fc1.weight")), gmp_u1)
    l2_prune = sws_prune_l2(dict((l, unified_model.state_dict()[l]) for l in ("fc2.bias", "fc2.weight")), gmp_u2)
    l3_prune = sws_prune_l2(dict((l, unified_model.state_dict()[l]) for l in ("fc3.bias", "fc3.weight")), gmp_u3)
    pruned_model = sws_replace(unified_model, [l1_prune, l2_prune, l3_prune])
    show_sws_weights(pruned_model)
    unify_res.data_prune(pruned_model)
    
    ures = unify_res.gen_dict()
    cm = compressed_model(pruned_model.state_dict(), [gmp_u1, gmp_u2, gmp_u3])
    ures['cm'] = cm.get_cr_list()
    ures['pre-sp'] = presparsity
    ures['pre-prune_acc'] = preprune

    show_sws_weights(pruned_model)
        
    ###SAVE MODEL, GMP, Res
    if (savedir != ""):
        #exp_name = "{}_m{}_zm{}_r{}_t{}_m{}_kdT{}_{}".format(unified_model.name, mean, zmean, retraining_epochs, tau, int(mixtures), int(temp), data_size)
        exp_name = "min_error3"
        torch.save(unified_model, savedir + '/mnist_retrain_layer_umodel_{}.m'.format(exp_name))
        with open(savedir + '/mnist_retrain_layer_u1gmp_{}.p'.format(exp_name),'wb') as f:
            pickle.dump(gmp_u1, f)
        with open(savedir + '/mnist_retrain_layer_u2gmp_{}.p'.format(exp_name),'wb') as f:
            pickle.dump(gmp_u2, f)
        with open(savedir + '/mnist_retrain_layer_u3gmp_{}.p'.format(exp_name),'wb') as f:
            pickle.dump(gmp_u3, f)
        with open(savedir + '/mnist_retrain_layer_ures_{}.p'.format(exp_name),'wb') as f:
            pickle.dump(ures, f)
    
    return layer_model_1, gmp_1, layer_model_2, gmp_2, layer_model_3, gmp_3, unified_model, ures

In [16]:
for tau in [2e-6, 1e-6, 8e-7, 5e-7, 3e-7, 1e-7]:
    print (tau)
    layer_model_1, gmp_1, res_1 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 1, "full", "LeNet_300_100", "MSEHNA", "./files")

2e-06


/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -2856.27
Epoch: 50. Loss: -2856.26
Original: 98.33% - Retrain: 97.99% - Prune: 91.80% - Quantize: 91.18% - Sparsity: 95.91%
1e-06
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -1428.05
Epoch: 50. Loss: -1428.02
Original: 98.33% - Retrain: 98.35% - Prune: 96.67% - Quantize: 96.63% - Sparsity: 92.91%
8e-07
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -1142.44
Epoch: 50. Loss: -1142.48
Original: 98.33% - Retrain: 98.34% - Prune: 96.98% - Quantize: 96.90% - Sparsity: 74.47%
5e-07
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -713.99
Epoch: 50. Loss: -713.99
Original: 98.33% - Retrain: 98.48% - Prune: 97.58% - Quantize: 97.48% - Sparsity: 76.91%
3e-07
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.

In [20]:
for tau in [2e-4, 1e-4, 8e-5, 5e-5, 3e-5, 1e-7]:
    print (tau)
    layer_model_2, gmp_2, res_2 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 2, "full", "LeNet_300_100", "MSEHNA", "./files")

0.0002
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0


/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Epoch: 25. Loss: -285650.97
Epoch: 50. Loss: -285651.25
Original: 98.33% - Retrain: 95.43% - Prune: 93.40% - Quantize: 93.23% - Sparsity: 98.21%
0.0001
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -142823.59
Epoch: 50. Loss: -142822.42
Original: 98.33% - Retrain: 97.00% - Prune: 95.63% - Quantize: 95.51% - Sparsity: 96.67%
8e-05
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -114258.34
Epoch: 50. Loss: -114257.51
Original: 98.33% - Retrain: 97.41% - Prune: 96.56% - Quantize: 96.38% - Sparsity: 95.87%
5e-05
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -71410.68
Epoch: 50. Loss: -71410.86
Original: 98.33% - Retrain: 97.77% - Prune: 97.35% - Quantize: 97.36% - Sparsity: 93.90%
3e-05
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -42846.04
Epoch: 50. Loss: -42845.98

In [23]:
for tau in [1e-5]:
    print (tau)
    layer_model_2, gmp_2, res_2 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 2, "full", "LeNet_300_100", "MSEHNA", "./files")

1e-05
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0


/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Epoch: 25. Loss: -14281.58
Epoch: 50. Loss: -14281.57
Original: 98.33% - Retrain: 98.27% - Prune: 98.28% - Quantize: 98.26% - Sparsity: 84.54%


In [22]:
for tau in [2e-2, 1e-2, 8e-3, 5e-3, 3e-3, 1e-3]:
    print (tau)
    layer_model_3, gmp_3, res_3 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 3, "full", "LeNet_300_100", "MSEHNA", "./files")

0.02
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0


/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Epoch: 25. Loss: -28565264.00
Epoch: 50. Loss: -28565266.00
Original: 98.33% - Retrain: 97.19% - Prune: 93.60% - Quantize: 93.79% - Sparsity: 78.51%
0.01
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -14282633.00
Epoch: 50. Loss: -14282633.00
Original: 98.33% - Retrain: 98.02% - Prune: 97.46% - Quantize: 97.46% - Sparsity: 81.49%
0.008
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -11426113.00
Epoch: 50. Loss: -11426112.00
Original: 98.33% - Retrain: 98.42% - Prune: 98.15% - Quantize: 98.13% - Sparsity: 73.07%
0.005
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -7141316.50
Epoch: 50. Loss: -7141315.50
Original: 98.33% - Retrain: 98.32% - Prune: 98.05% - Quantize: 98.04% - Sparsity: 76.63%
0.003
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -4284789.00
Epoch: 50.

In [24]:
unified_model = sws_replace(model_archs.LeNet_300_100().cuda(), [layer_model_1.state_dict(), layer_model_2.state_dict(), layer_model_3.state_dict()])
test_acc = test_accuracy(test_data_full, test_labels_full, unified_model)

In [ ]:
for tau in [2e-6, 1e-6, 8e-7, 5e-7, 3e-7, 1e-7]:
    print (tau)
    layer_model_1, gmp_1, res_1 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 1, "full", "LeNet_300_100", "MSEHA", "./mseha")

for tau in [2e-4, 1e-4, 8e-5, 5e-5, 3e-5, 1e-5]:
    print (tau)
    layer_model_2, gmp_2, res_2 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 2, "full", "LeNet_300_100", "MSEHA", "./mseha")
    
for tau in [2e-2, 1e-2, 8e-3, 5e-3, 3e-3, 1e-3]:
    print (tau)
    layer_model_3, gmp_3, res_3 = retrain_layer(5000, 2, 5000, 10, 16, 1, tau, 3, "full", "LeNet_300_100", "MSEHA", "./mseha")
    


2e-06


/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'model_archs.LeNet_300_100' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -2844.18
Epoch: 50. Loss: -2842.71
Original: 98.33% - Retrain: 94.68% - Prune: 65.06% - Quantize: 69.47% - Sparsity: 98.54%
1e-06
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -1416.57
Epoch: 50. Loss: -1414.18
Original: 98.33% - Retrain: 97.24% - Prune: 96.06% - Quantize: 96.00% - Sparsity: 88.06%
8e-07
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -1129.57
Epoch: 50. Loss: -1129.65
Original: 98.33% - Retrain: 97.52% - Prune: 96.50% - Quantize: 95.34% - Sparsity: 78.47%
5e-07
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.0 Variance: 2.0
Epoch: 25. Loss: -700.02
Epoch: 50. Loss: -700.86
Original: 98.33% - Retrain: 98.02% - Prune: 97.40% - Quantize: 97.44% - Sparsity: 81.12%
3e-07
0-component Mean: 5000.0 Variance: 10.0
Non-zero component Mean: 5000.